# Neurosymbolic Reasoner - Google Colab Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChessEngineUS/neurosymbolic-reasoner/blob/main/neurosymbolic_colab_demo.ipynb)

This notebook demonstrates the complete neurosymbolic AI system running on Google Colab's T4 GPU.

**Features:**
- Neural perception with transformer encoders
- Symbolic reasoning with logic engines
- Integrated training and inference
- T4 GPU optimization

## 1. Setup and Installation

First, let's check our GPU and install dependencies.

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Clone the repository
!git clone https://github.com/ChessEngineUS/neurosymbolic-reasoner.git
%cd neurosymbolic-reasoner

In [ ]:
# Install dependencies
!pip install -q torch>=2.0.0 numpy scipy transformers accelerate tqdm matplotlib

## 2. Import and Initialize System

In [ ]:
import sys
import torch
import numpy as np
from neurosymbolic import NeurosymbolicSystem, Predicate, Rule

print("Successfully imported neurosymbolic system!")

In [ ]:
# Initialize the neurosymbolic system
print("Initializing neurosymbolic system...")

system = NeurosymbolicSystem(
    input_dim=512,
    hidden_dim=768,
    num_concepts=50,
    num_predicates=30
)

# Optimize for T4 GPU
system.optimize_for_t4()

print(f"✓ System initialized on {system.device}")

## 3. Add Symbolic Knowledge

Let's build a knowledge base with facts and rules.

In [ ]:
# Define knowledge base
knowledge = {
    'facts': [
        {'name': 'mammal', 'arity': 1, 'args': ['dog']},
        {'name': 'mammal', 'arity': 1, 'args': ['cat']},
        {'name': 'mammal', 'arity': 1, 'args': ['whale']},
        {'name': 'has_fur', 'arity': 1, 'args': ['dog']},
        {'name': 'has_fur', 'arity': 1, 'args': ['cat']},
        {'name': 'lives_in_water', 'arity': 1, 'args': ['whale']},
        {'name': 'has_legs', 'arity': 1, 'args': ['dog']},
        {'name': 'has_legs', 'arity': 1, 'args': ['cat']}
    ],
    'rules': [
        {
            'premises': [
                {'name': 'mammal', 'arity': 1, 'args': ['?x']},
                {'name': 'has_fur', 'arity': 1, 'args': ['?x']}
            ],
            'conclusion': {'name': 'warm_blooded', 'arity': 1, 'args': ['?x']},
            'confidence': 0.95
        },
        {
            'premises': [
                {'name': 'has_legs', 'arity': 1, 'args': ['?x']},
                {'name': 'has_fur', 'arity': 1, 'args': ['?x']}
            ],
            'conclusion': {'name': 'land_animal', 'arity': 1, 'args': ['?x']},
            'confidence': 0.90
        }
    ],
    'predicate_map': {
        0: 'mammal',
        1: 'has_fur',
        2: 'warm_blooded',
        3: 'has_legs',
        4: 'land_animal',
        5: 'lives_in_water'
    }
}

system.add_knowledge(knowledge)
print("✓ Knowledge base loaded")
print(f"  Facts: {len(knowledge['facts'])}")
print(f"  Rules: {len(knowledge['rules'])}")
print(f"  Predicates: {len(knowledge['predicate_map'])}")

## 4. Neural Perception

Process perceptual input through the neural module.

In [ ]:
# Generate sample perceptual input
batch_size = 8
seq_len = 16
input_dim = 512

print(f"Generating input: {batch_size} samples, {seq_len} sequence length")
input_data = torch.randn(batch_size, seq_len, input_dim)

if torch.cuda.is_available():
    input_data = input_data.cuda()

# Process through neural module
print("\nProcessing through neural encoder...")
with torch.no_grad():
    neural_output = system.neural_module(input_data)

print("✓ Neural processing complete")
print(f"  Encoded shape: {neural_output['encoded'].shape}")
print(f"  Concept probabilities shape: {neural_output['concept_probs'].shape}")
print(f"  Top-3 activated concepts: {neural_output['concept_probs'][0].topk(3).indices.tolist()}")

## 5. Symbolic Reasoning

Demonstrate logical inference with the symbolic module.

In [ ]:
# Test reasoning queries
queries = [
    {'name': 'warm_blooded', 'arity': 1, 'args': ['dog']},
    {'name': 'warm_blooded', 'arity': 1, 'args': ['cat']},
    {'name': 'land_animal', 'arity': 1, 'args': ['dog']},
    {'name': 'mammal', 'arity': 1, 'args': ['whale']}
]

print("Testing reasoning queries:\n")
for i, query in enumerate(queries, 1):
    result = system.symbolic_module.reason(query, method='forward')
    pred = Predicate(**query)
    print(f"{i}. Query: {pred}")
    print(f"   Answer: {result['answer']}")
    if 'confidence' in result:
        print(f"   Confidence: {result['confidence']:.2f}")
    print()

In [ ]:
# Generate explanation
query = {'name': 'warm_blooded', 'arity': 1, 'args': ['dog']}
explanation = system.symbolic_module.explain(query)

print("Reasoning Explanation:")
print("=" * 50)
print(explanation)

## 6. Integrated Perception + Reasoning

Combine neural perception with symbolic reasoning.

In [ ]:
# Run integrated system
print("Running integrated neurosymbolic pipeline...\n")

input_data = torch.randn(4, 16, 512)
if torch.cuda.is_available():
    input_data = input_data.cuda()

query = {'name': 'warm_blooded', 'arity': 1, 'args': ['dog']}
result = system.perceive_and_reason(input_data, query=query)

print("Results:")
print("=" * 50)
print(f"Extracted predicates: {len(result['extracted_predicates'])}")

for i, pred in enumerate(result['extracted_predicates'][:5], 1):
    pred_name = knowledge['predicate_map'].get(pred['predicate_id'], 'unknown')
    print(f"{i}. {pred_name} - confidence: {pred['confidence']:.3f}")

if result['reasoning_result']:
    print(f"\nReasoning Answer: {result['reasoning_result']['answer']}")
    if 'confidence' in result['reasoning_result']:
        print(f"Confidence: {result['reasoning_result']['confidence']:.3f}")

## 7. Training Demo

Quick training demonstration with synthetic data.

In [ ]:
import torch.optim as optim
from tqdm import tqdm

# Generate synthetic training data
def generate_data(num_samples=100):
    inputs = torch.randn(num_samples, 16, 512)
    labels = torch.zeros(num_samples, 50)
    for i in range(num_samples):
        num_active = torch.randint(1, 6, (1,)).item()
        active_concepts = torch.randperm(50)[:num_active]
        labels[i, active_concepts] = 1.0
    return inputs, labels

print("Generating training data...")
train_inputs, train_labels = generate_data(200)

# Setup optimizer
optimizer = optim.AdamW(
    list(system.neural_module.parameters()) + list(system.bridge.parameters()),
    lr=1e-4
)

# Training loop
print("\nTraining for 5 epochs...")
batch_size = 16
num_epochs = 5

for epoch in range(num_epochs):
    total_loss = 0
    num_batches = 0
    
    for i in range(0, len(train_inputs), batch_size):
        batch_inputs = train_inputs[i:i+batch_size].to(system.device)
        batch_labels = train_labels[i:i+batch_size].to(system.device)
        
        labels_dict = {'concepts': batch_labels}
        losses = system.train_step(batch_inputs, labels_dict, optimizer)
        
        total_loss += losses['total_loss']
        num_batches += 1
    
    avg_loss = total_loss / num_batches
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f}")

print("\n✓ Training complete!")

## 8. Performance Benchmark

Measure inference speed on T4 GPU.

In [ ]:
import time

# Benchmark inference speed
batch_sizes = [4, 8, 16, 32]
num_runs = 50

print("Benchmarking inference speed...\n")
print(f"{'Batch Size':<12} {'Avg Time (ms)':<15} {'Throughput (samples/s)'}")
print("-" * 50)

system.neural_module.eval()

for bs in batch_sizes:
    times = []
    
    # Warmup
    for _ in range(5):
        test_input = torch.randn(bs, 16, 512).to(system.device)
        with torch.no_grad():
            _ = system.neural_module(test_input)
    
    # Benchmark
    for _ in range(num_runs):
        test_input = torch.randn(bs, 16, 512).to(system.device)
        
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        
        start = time.time()
        with torch.no_grad():
            _ = system.neural_module(test_input)
        
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        
        times.append(time.time() - start)
    
    avg_time = np.mean(times) * 1000  # Convert to ms
    throughput = bs / (avg_time / 1000)
    
    print(f"{bs:<12} {avg_time:<15.2f} {throughput:.1f}")

print("\n✓ Benchmark complete!")

## 9. Summary

This notebook demonstrated:

1. ✅ Neural perception with transformers
2. ✅ Symbolic reasoning with logic engines
3. ✅ Integrated neurosymbolic inference
4. ✅ Training pipeline
5. ✅ T4 GPU optimization
6. ✅ Performance benchmarking

**Next Steps:**
- Customize the knowledge base for your domain
- Add vision encoders for image inputs
- Integrate with NLP models for text reasoning
- Scale up training with larger datasets

**Repository:** https://github.com/ChessEngineUS/neurosymbolic-reasoner